# 1. Extract

In [8]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
import glob
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from io import StringIO
import shutil

AttributeError: module 'numpy.random' has no attribute 'BitGenerator'

In [ ]:
def read_csv_folder(folder_path):
    """
    Read CSV files from a folder and create a DataFrame for each file.

    Parameters:
        folder_path (str): The path to the folder containing the CSV files.

    Returns:
        dict: A dictionary containing the file name as key and a DataFrame containing the data from the CSV file as value.
    """
    dataframes = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            filepath = os.path.join(folder_path, filename)
            df = pd.read_csv(filepath)
            dataframes[filename] = df
    return dataframes

## 1.1 Unemployment
- Portal datos Abiertos Gobierno España

In [ ]:
PATH=ChromeDriverManager().install()

In [ ]:
def extract_selenium(url, texto, name):
    driver = webdriver.Chrome(PATH)
    driver.get(portal_gob)
    try:
        catalogo_input=driver.find_element_by_xpath('//*[@id="search_bar"]')
        catalogo_input.send_keys(texto)
        catalogo=driver.find_element_by_xpath('//*[@id="search"]')
        catalogo.click()
        time.sleep(2)
        paro_pagina= driver.find_element_by_xpath('/html/body/div[3]/main/div/div/div/div[3]/div/section/div[1]/div/ul/li[1]/div/strong')
        paro_pagina.click()
        time.sleep(2)
        paro_link= driver.find_element_by_xpath('/html/body/div[3]/main/div/div/div/div[3]/div/article/div/section[4]/ul/li[40]/div[5]/a')
        paro_link.click()
        time.sleep(2)
        list_of_files = glob.glob('/downloads/*.csv') # * means all if need specific format then *.csv
        latest_file = max(list_of_files, key=os.path.getctime)
        destination = '/data'
        new_file_name = 'Paro_municipios_2019.csv'
        shutil.move(latest_file, destination)
        os.rename(destination, new_file_name)
        print('Moved to location done')
    except:
        print('Error')
    return 'Completed with no errors'

In [ ]:
portal_gob = 'https://datos.gob.es/es/catalogo'
texto = 'Paro Registrado por municipios'
name = 'Paro_municipios_2019'

In [ ]:
extract_selenium(portal_gob,texto,name)

----------------------------

## 1.2 Income Tax
- Selenium

In [ ]:
url_scraping= "https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfmunicipios/2019/jrubikf74b3dca9af01b51cabd6d5603e0e16daecd1a97c.html"

In [ ]:
html=req.get(url_scraping).content

soup=bs(html, 'html.parser')

tabla=soup.find_all('table')  # find_all(tag)

In [ ]:
columnas=[e.text.strip() for e in tabla[0].find_all('th')]

In [ ]:
#### Get namecols "RENTA BRUTA MEDIA , RENTA DISPONIBLE MEDIA'
cols = columnas[6:8]
cols_ccaa = [item.replace('RENTA BRUTA MEDIA','renta_bruta_media').replace('RENTA DISPONIBLEMEDIA', 'renta_disponible_media') for item in cols]
cols_ccaa.insert(0, "ccaa")
cols_provincias=[item.replace('RENTA BRUTA MEDIA','renta_bruta_media').replace('RENTA DISPONIBLEMEDIA', 'renta_disponible_media') for item in cols]
cols_provincias.insert(0, "provincia")

In [ ]:
#### Get CCAA Data (except Pais Vasco and Navarra)
#Get data for CCAA
for table in soup.find_all("tr", class_="depth_1"):
    ccaa.append(table.find_all('th')[0].text)
    rb=table.find('td', {'headers': 'table01_c6'})
    ccaa_renta_bruta_media.append(rb.text)
    rm=table.find('td', {'headers': 'table01_c7'})
    ccaa_renta_disponible_media.append(rm.text)

In [ ]:
#Erase characters
ccaa=[el.replace(u'\xa0','') for el in ccaa]
ccaa=[el.strip() for el in ccaa]

In [ ]:
### Get data from provinces
provinces_table = soup.findAll(class_= 'depth_2') 
provincias_renta_bruta_media = []
provincias_renta_disponible_media = []
provincias=[]

In [ ]:
for table in soup.find_all('tr', class_='depth_2'):
    provincias.append(table.find_all('th')[0].text)
    provincias_renta_bruta_media.append(table.find('td', {'headers': 'table01_c6'}).text)
    provincias_renta_disponible_media.append(table.find('td', {'headers': 'table01_c7'}).text)

In [ ]:
#Erase characters
provincias=[el.replace(u'\xa0','') for el in provincias]
provincias=[el.strip() for el in provincias]

#### Get Pais Vasco data

In [ ]:
url_PV = 'https://www.eustat.eus/elementos/ele0005700/renta-personal-media-de-la-c-a-de-euskadi-por-ambitos-territoriales-segun-tipo-de-renta-euros/tbl0005790_c.html'

In [ ]:
html=req.get(url_PV).content

soup=bs(html, 'html.parser')

tabla_e=soup.find_all('table')

In [ ]:
for i,table in enumerate(soup.find_all("tr", class_="Nivel3B")):
    if i==0:
        ccaa.append(table.find('td').text)
        ccaa_renta_bruta_media.append(table.find('td', {'class': 'td8Bold'}).text)
        ccaa_renta_disponible_media.append(table.find_all('td', {'class': 'td8Bold'})[-1].text)

    elif i==1:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)

    elif i==2:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)
    
    elif i==3:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)

In [ ]:
ccaa[-1]='Pais Vasco'
ccaa=[el.replace(u'\xa0','') for el in ccaa]
ccaa=[el.strip() for el in ccaa]
provincias=[el.replace(u'\xa0','') for el in provincias]
provincias=[el.strip() for el in provincias]

#### Get Navarra data

In [ ]:
ccaa.insert(20,'Navarra')
ccaa_renta_disponible_media.append(13.394)
ccaa_renta_bruta_media.append(16.300)

In [ ]:
provincias.insert(53,'Navarra')
provincias_renta_bruta_media.append(13.394)
provincias_renta_disponible_media.append(16.300)

## OUTPUT DATA

In [ ]:
df_ccaa_renta = pd.DataFrame(zip(ccaa, ccaa_renta_bruta_media,ccaa_renta_disponible_media),
               columns =[cols_ccaa])

In [ ]:
df_provincias_renta = pd.DataFrame(zip(provincias, provincias_renta_bruta_media,provincias_renta_disponible_media),
               columns =[cols_provincias])

In [ ]:
df_ccaa_renta.to_csv('../data/ccaa_renta.csv', encoding='latin1', index=False)

In [ ]:
df_provincias_renta.to_csv('../data/provincias_renta.csv', encoding='latin1', index=False)